In [1]:
import os
from music21 import converter, pitch, interval, instrument, note, chord
import numpy as np
import json
import tensorflow as tf

data = './data/'

if not os.path.exists(data):
    os.makedirs(data)

2022-11-18 12:48:20.274264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 12:48:20.464695: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 12:48:21.312983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-18 12:48:21.313056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
songList = [song for song in os.listdir(data) if song.lower().find('giuliani')>-1]
print(len(songList) , 'songs found by Giuliani')

88 songs found by Giuliani


In [3]:
def monophonic(stream):
    return len(instrument.partitionByInstrument(stream).parts) == 1

songList = [song for song in songList if monophonic(converter.parse(data+song))]
print(len(songList) , 'monophonic songs found by Giuliani')

32 monophonic songs found by Giuliani


In [4]:
if not os.path.exists('./giuliani-monophonic/'):
    os.makedirs('./giuliani-monophonic/')

if len(os.listdir('./giuliani-monophonic/')) == 0:
    for song in songList:
        os.system('cp ' + data + song + ' ./giuliani-monophonic/' + song)

In [5]:
streams = [converter.parse('./giuliani-monophonic/' + song) for song in songList]
chordifiedStreams = [stream.chordify() for stream in streams]

In [6]:
originalChords = [[] for _ in chordifiedStreams]
originalDurations = [[] for _ in chordifiedStreams]
originalKeys = []

for i, song in enumerate(chordifiedStreams):
    originalKeys.append(str(song.analyze('key')))

    for measure in song.recurse().getElementsByClass('Measure'):
        for element in measure:
            if isinstance(element, chord.Chord):
                originalChords[i].append('.'.join(str(n) for n in element.pitches))
                originalDurations[i].append(element.duration.quarterLength)

print('Original chords: ', originalChords[0][:10])

Original chords:  ['D5', 'G3.D5', 'B3.D5', 'D4.D5', 'G3.B4', 'B3.B4', 'D4.B4', 'G3.G4', 'B3.G4', 'D4.G4']


In [7]:
print(originalKeys)

mostCommonKey = max(set(originalKeys), key=originalKeys.count)

print(mostCommonKey , 'is the most common key with' , originalKeys.count(mostCommonKey) , 'occurrences')

['G major', 'C major', 'G major', 'a minor', 'C major', 'a minor', 'd minor', 'D major', 'C major', 'G major', 'G major', 'C major', 'a minor', 'A major', 'A major', 'C major', 'D major', 'C major', 'D major', 'D major', 'C major', 'C major', 'A major', 'a minor', 'G major', 'A major', 'a minor', 'G major', 'C major', 'C major', 'a minor', 'g minor']
C major is the most common key with 10 occurrences


In [8]:
mckChords = []
mckDurations = []

for key in originalKeys:
    if key == mostCommonKey:
        mckChords.append(originalChords[originalKeys.index(key)])
        mckDurations.append(originalDurations[originalKeys.index(key)])

print(len(mckChords) , 'songs in the most common key')

10 songs in the most common key


In [9]:
uniqueChords = np.unique([i for s in originalChords for i in s])
chordToInt = dict(zip(uniqueChords, list(range(0, len(uniqueChords)))))

uniqueDurations = np.unique([i for s in originalDurations for i in s])
durationToInt = dict(zip(uniqueDurations, list(range(0, len(uniqueDurations)))))

intToChord = {i: c for c, i in chordToInt.items()}
intToDuration = {i: c for c, i in durationToInt.items()}

In [10]:
# Set sequence length
sequenceLength = 32

# Define empty array for train data
trainChords = []
trainDurations = []
targetChords = []
targetDurations = []

# Construct train and target sequences for chords and durations
for s in range(len(mckChords)):
    chordList = [chordToInt[c] for c in mckChords[s]]
    durationList = [durationToInt[d] for d in mckDurations[s]]
    for i in range(len(chordList) - sequenceLength):
        trainChords.append(chordList[i:i+sequenceLength])
        trainDurations.append(durationList[i:i+sequenceLength])
        targetChords.append(chordList[i+1])
        targetDurations.append(durationList[i+1])


In [11]:
# Define number of samples, notes and chords, and durations
nSamples = trainChords.shape[0]
nChords = trainChords.shape[1]
nDurations = trainDurations.shape[1]

# Set the input dimension
inputDim = nChords * sequenceLength

# Set the embedding layer dimension
embedDim = 64


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Define input layers
chordInput = tf.keras.layers.Input(shape = (None,))
durationInput = tf.keras.layers.Input(shape = (None,))

# Define embedding layers
chordEmbedding = tf.keras.layers.Embedding(nChords, embedDim, input_length = sequenceLength)(chordInput)
durationEmbedding = tf.keras.layers.Embedding(nDurations, embedDim, input_length = sequenceLength)(durationInput)

# Merge embedding layers using a concatenation layer
mergeLayer = tf.keras.layers.Concatenate(axis=1)([chordEmbedding, durationEmbedding])

# Define LSTM layer
lstmLayer = tf.keras.layers.LSTM(512, return_sequences=True)(mergeLayer)

# Define dense layer
denseLayer = tf.keras.layers.Dense(256)(lstmLayer)

# Define output layers
chordOutput = tf.keras.layers.Dense(nChords, activation = 'softmax')(denseLayer)
durationOutput = tf.keras.layers.Dense(nDurations, activation = 'softmax')(denseLayer)

# Define model
lstm = tf.keras.Model(inputs = [chordInput, durationInput], outputs = [chordOutput, durationOutput])


In [ ]:
# Compile the model
lstm.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Train the model
lstm.fit([trainChords, trainDurations], [targetChords, targetDurations],
            epochs=500, batch_size=64)


In [ ]:
# Define initial chord and duration sequences
initialChords = np.expand_dims(trainChords[0,:].copy(), 0)
initialDurations = np.expand_dims(trainDurations[0,:].copy(), 0)

# Define function to predict chords and durations
def predictChords(chordSequence, durationSequence):
    predictedChords, predictedDurations = model.predict(model.predict([chordSequence, durationSequence]))
    return np.argmax(predictedChords), np.argmax(predictedDurations)

# Define empty lists for generated chords and durations
newChords, newDurations = [], []

# Generate chords and durations using 500 rounds of prediction
for j in range(500):
    newChord, newDuration = predictChords(initialChords, initialDurations)
    newChords.append(newChord)
    newDurations.append(newDuration)
    initialChords[0][:-1] = initialChords[0][1:]
    initialChords[0][-1] = newChord
    initialDurations[0][:-1] = initialDurations[0][1:]
    initialDurations[0][-1] = newDuration


In [ ]:
# Create stream object and add guitar as instrument
generatedStream = stream.Stream()
generatedStream.append(instrument.Guitar())

# Add notes and durations to stream
for j in range(len(chordSequence)):
    try:
        generatedStream.append(note.Note(chordSequence[j].replace('.', ' '), quarterType = durationSequence[j]))
    except:
        generatedStream.append(chord.Chord(chordSequence[j].replace('.', ' '), quarterType = durationSequence[j]))

# Export as MIDI file
generatedStream.write('midi', fp=generated_dir+'lstm.mid')
